# 4.2. Passives Reinforcement-Learning

>## <ins>Table of contents</ins> <a name="up"></a>[<sup>[1]</sup>](#cite_note-1)
>* [**4.2.1. Probeläufe und Zustandsnutzen**](#4_2_1)
>* [**4.2.2. Adaptive dynamische Programmierung**](#4_2_2)
>* [**4.2.3. Temporal Difference Learning**](#4_2_3)
>* [**4.1.4. Glossar**](#4_2_4)
>
>## <ins>Beispiele</ins>
>* [**Beispiel 1**: Staubsauger](#b1)
>* [**Beispiel 2**: Multimenge der Probeläufen und der Nutzen $U^{\gamma}_{\partial}(s | \pi)$](#b2)
>* [**Beispiel 3**: Nutzen einer Episode $e$ *Staubsauger*](#b3)
>* [**Beispiel 4**: Nutzen einer Strategie $\pi$ von *Staubsauger*](#b4)


## Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#### **Problemstellung**:

- Die im vorherigen Unterkapitel diskutierten Methoden lernen eine optimale Strategie $\pi^*$, vorausgesetzt, es gibt ein korrektes Modell der Umgebung in Form eines Markov-Entscheidungsprozesses. In den meisten realistischen Anwendungsszenarien existiert kein solches Modell. Beispielsweise hat ein Staubsaugerroboter normalerweise keinen Lageplan der zu reinigenden Wohnung und ein Schachspieler kein Modell der gegnerischen Strategie.

- Ein weiterer Nachteil der Methoden des vorherigen Unterkapitels ist ihre hohe Lernzeit. Zum Beispiel hat Schach ungefähr $10^{45}$ Zustände und eine einzige Iteration des SI-Algorithmus müsste bereits die Nutzenwerte aller dieser Zustände in Bezug auf eine initiale Strategie approximieren.
  
- Im Folgenden werden Ansätze diskutiert, <ins>die eine optimale Strategie erlernen</ins>, ohne ein Modell der Umgebung zu nutzen. Außerdem werden Methoden des passiven Reinforcement Learnings betrachtet, d.h., Methoden, die bereits eine festgelegte Strategie annehmen und ausschließlich die Nutzenwerte der Zustände erlernen (die dann auch zur Verbesserung der Strategie genutzt werden können).
- Im nächsten Unterkapitel wird aktives Reinforcement Learning betrachtet, bei dem während des Lernens der Nutzenwerte auch direkt die optimale Strategie erlernt wird.
  
## 4.2.1. Probeläufe und Zustandsnutzen <a name="4_2_1"></a>
---

- Sei $π$ eine Strategie.
- Imm Algemeinen, ist das Ziel der folgenden Methoden die Bestimmung des Nutzens $U^{\gamma}_D(s | \pi)$ aller Zustände $s \in S \setminus St$ bezüglich π eines unbekannten Markov-Entscheidungsprozesses $D = (S,A,P,R,s_0,St)$ und mit einen gegebenen Diskontfaktor $γ$.

- <a name="U_gamma_D"></a> Nach Definition 6 aus Unterkapitel 4.1 ist $U^{\gamma}_D(s | \pi)$ definiert als $$U^{\gamma}_D(s | \pi) = \sum_{\pi \sim e=(s,a_1,s_1,...)} P(e) \sum_i \gamma^{i-1} R(s_{i-1},a_i,s_i)$$ 
```

Wenn Sie auf "Google" klicken, werden Sie zur Google-Homepage weitergeleitet. Bitte beachten Sie, dass es keine Leerzeichen zwischen den eckigen und den runden Klammern geben sollte.

- Die **Aufgabe des passiven Reinforcement Learnings** entspricht der Strategieevaluation aus Abschnitt 4.1.3 (*Policy evaluation*), mit dem Unterschied, dass die Effekte von Aktionen (insbesondere die Übergangswahrscheinlichkeiten *P*) zunächst nicht bekannt sind.
- Um $U^{\gamma}_D(s | \pi)$ zu lernen, führt der Agent eine Anzahl von "Probefahrten" in der Umgebung aus und protokolliert sowohl die besuchten Zustände als auch die erhaltenen Belohnungen.

> **Definition 1. Beobachtung $o$ und Probelauf $O$**
>
> Sei $\pi : S \rightarrow A$ eine Strategie bzgl. einer Menge von Zuständen $S$ und einer Menge von Aktionen $A$.
>
> Eine Beobachtung $o$ ist ein Tupel $o = (s,a,s',r)$ mit $s,s' \in S$, $a \in A$, $r \in R$.
>
> Eine Beobachtung $o = (s,a,s',r)$ ist $\pi$-induziert, falls $a = \pi(s)$. Ein Probelauf $O$ bzgl.
>
> $\pi$ ist eine (endliche) Sequenz $O = (o_1,...,o_n)$ mit $\pi$-induzierten Beobachtungen $o_i = (s_i,a_i,s'_i,r_i)$ mit $s'_i = s_{i+1}$, für $i = 1,...,n-1$. Für $\gamma \in [0,1]$ definiere
$$rew^{\gamma}(O) = \sum_{i=1}^{n} \gamma^{i-1}r_i$$

Die Definition beschreibt die Konzepte der Beobachtung und des Probelaufs in einem Entscheidungsprozess, der durch eine Strategie $\pi$ gesteuert wird.

- **Beobachtung ($o$)**: Ein Tupel bestehend aus einem Zustand $s$, einer Aktion $a$, einem Folgezustand $s'$ und einer Belohnung $r$. Eine Beobachtung ist $\pi$-induziert, wenn die Aktion $a$ gleich der Aktion ist, die von der Strategie $\pi$ für den Zustand $s$ ausgewählt wurde.

- **Probelauf ($O$)**: Eine Sequenz von $\pi$-induzierten Beobachtungen D.h.  unter Ausführung der Strategie $π$. Jede Beobachtung in der Sequenz besteht aus einem Zustand $s_i$, einer Aktion $a_i$, einem Folgezustand $s'_i$ und einer Belohnung $r_i$. Der Folgezustand einer Beobachtung ist der Zustand der nächsten Beobachtung in der Sequenz.

  **!!** Beachten Sie, dass ein Probelauf eine direkte Entsprechung zu einer (initialen) Episode hat, der Unterschied ist nur dass ein Probelauf zusätzlich Informationen zu Belohnungen enthält.

- **Belohnungsfunktion ($rew^\gamma$)**: Eine Funktion, die eine Sequenz von Beobachtungen nimmt und eine gewichtete Summe der Belohnungen bzw. die diskontierte Summe der Belohnungen für einen Probelauf $O$ berechnet. Jede Belohnung wird mit $\gamma^{i-1}$ gewichtet, wobei $i$ der Index der Beobachtung in der Sequenz ist und $\gamma$ ein Diskontierungsfaktor zwischen 0 und 1 ist. Diese Funktion wird verwendet, um die Gesamtbelohnung eines Probelaufs zu berechnen. Je größer der Diskontierungsfaktor $\gamma$ (Je näher an 1), desto stärker werden zukünftige Belohnungen gewichtet. Wenn $\gamma = 0$, zählt nur die sofortige Belohnung.

#### `Die Lernaufgabe -->` ist aus einer Reihe von Probelaufen $O_1,...,O_m$, sollen die Werte $U^{\gamma}_D(s | \pi)$ für alle $s \in S \setminus St$ approximiert werden.

#### **Beispiel 1.** <a name="b1"></a> Fortsetzung des Staubsaugerroboterproblems

Wir betrachten wieder das Beispiel des Staubsaugerroboters aus [Unterkapitel 4.1.](http://localhost:8888/lab/tree/KE4_Reinforcement_Learning/4.1.-Markov-Entscheidungsprozess.ipynb). In diesem Beispiel geht es um einen Staubsaugerroboter, der in einer Umgebung mit zwei Räumen (Raum r1 und Raum r2) agiert. Der Roboter kann drei Aktionen ausführen: `clean` (reinigen), `move` (bewegen) und `charge` (aufladen).

Die Zustände des Roboters sind durch die Menge $S_{vc}$ repräsentiert:

$$
S_{vc} = \{s_{1,1}^{1}, s_{1,1}^{2}, s_{0,1}^{1}, s_{0,1}^{2}, s_{1,0}^{1}, s_{1,0}^{2}, s_{0,0}^{1}, s_{0,0}^{2}, s_{t}\}
$$

Diese Zustände beinhalten verschiedene Kombinationen von Raumposition und Raumreinigungsstatus. Zum Beispiel repräsentiert der Zustand $s_{1,1}^{1}$, dass der Roboter sich in Raum r1 befindet, und Beide Räume sind sauber. Der Zustand $s_{t}$ repräsentiert einen Terminalzustand, in dem der Roboter seine Aufgabe abgeschlossen hat.

Die Menge der Aktionen $A_{vc}$ ist

$$
A_{vc} = \{\text{move}, \text{clean}, \text{charge}\}
$$

Wir betrachten weiterhin die Strategie $\pi_{vc}$ mit

$$\pi_{vc}(s_{1,1}^{1}) = \pi_{vc}(s_{1,0}^{1}) = \pi_{vc}(s_{1,1}^{2}) = \pi_{vc}(s_{0,1}^{2}) = \text{clean} $$
$$\pi_{vc}(s_{0,1}^{1}) = \pi_{vc}(s_{1,0}^{2}) = \pi_{vc}(s_{0,0}^{2}) = \text{move}$$
$$\pi_{vc}(s_{0,0}^{1}) = \text{charge}$$

$\pi_{vc}$ definiert, welche Aktion der Roboter in jedem Zustand ausführen soll. Zum Beispiel führt der Roboter die Aktion `clean` aus, wenn er sich im Zustand $s_{1,1}^{1}$, $s_{1,0}^{1}$, $s_{1,1}^{2}$ oder $s_{0,1}^{2}$ befindet.

und die folgenden drei Probelaufe

$$
O1 = ((s_{1,1}^{1}, \text{clean}, s_{0,1}^{1}, 10), (s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)) $$
$$O2 = ((s_{1,1}^{1}, \text{clean}, s_{1,1}^{1}, 0), (s_{1,1}^{1}, \text{clean}, s_{0,1}^{1}, 10), (s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)) $$
$$O3 = ((s_{1,1}^{1}, \text{clean}, s_{0,1}^{1}, 10), (s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0))
$$

Die Probelaufe $O1$, $O2$ und $O3$ sind Sequenzen von Zuständen und Aktionen, die der Roboter durchläuft, wobei Probelauf $O1$ entspricht dem erwarteten, idealen Ablauf (der Roboter reinigt zunächst Raum $r1$, bewegt sich anschließend in Raum $r2$, reinigt diesen, kehrt zurück in Raum $r1$, und lädt sich auf). Die Probelaufe $O2$ und $O3$ enthalten dagegen einige fehlgeschlagene Aktionen.

Nach der Definition von [$U^{\gamma}_D(s | \pi)$](#U_gamma_D) ist der Nutzen eines Zustands das gewichtete Mittel aller Episoden, die in diesem Zustand starten, gemäß einer bestimmten Strategie $\pi$. 

Wenn ein Zustand $s$ in einem Probelauf $O$ auftritt, enthält $O$ als Teilsequenz eine Episode, die in $s$ startet. Es kann auch mehrere solcher Episoden geben, wenn $s$ mehrmals in $O$ besucht wurde. Eine Episode, die mit dem Zustand $s_{1,1}^{1}$ beginnt und als Teilsequenzen in $O_1$ enthalten ist, könnte wie folgt aussehen:

$$
(s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)
$$

Der Nutzen $U^{\gamma}_D(s | \pi)$ kann dann durch den Durchschnitt der Nutzen dieser beobachteten Teilsequenzen approximiert werden. 

Das bedeutet, dass der erwartete Nutzen eines Zustands über alle möglichen Episoden, die von diesem Zustand ausgehen, gemittelt wird. Dies berücksichtigt sowohl die unmittelbaren Belohnungen als auch die zukünftigen Belohnungen, die durch das Ausführen der Strategie $\pi$ ab diesem Zustand erzielt werden können. 

> **Definition 2. Multimenge der Probeläufen und der Nutzen $U^{\gamma}_{\partial}(s | \pi)$**
>
> Sei $O$ ein Probelauf mit $O = (o_1,...,o_n)$, eine Sequenz von Beobachtungen $o_i = (s_i,a_i,s'_i,r_i)$ (für $i = 1,...,n$) und $s \in S \setminus S^t$ ein Zustand. Definiere die Menge $O_s$ durch
$$
O_s = \{(o_k,...,o_n) | s_k = s\}
$$
Für eine (Multi-)Menge $\vartheta = \{O^1,...,O^m\}$ an Probelaufen definiere die Multimenge
$$
\partial_s = O_s^1 \cup ... \cup O_s^m
$$
und
$$
U^\gamma_{\partial}(s | \pi) = \frac{\sum_{O \in \partial_s} \text{rew}^\gamma (O)}{|\partial_s|}
$$


Die Menge $O_s$ ist definiert als die Menge aller Teilsequenzen von $O$, die mit dem Zustand $s$ beginnen. Das heißt, für jeden Probelauf in $O$, wenn der Zustand $s$ erreicht wird, wird die verbleibende Sequenz von Ereignissen als Teil von $O_s$ betrachtet.

Wenn wir eine Menge von Probeläufen $O = \{O_1, ..., O_m\}$ haben, dann ist die Multimenge $\partial_s$ definiert als die Vereinigung der Mengen $O_s^1, ..., O_s^m$, die jeweils die Teilsequenzen von $O_1, ..., O_m$ sind, die mit dem Zustand $s$ beginnen.

Schließlich ist der Nutzen $U^{\gamma}_{\partial}(s | \pi)$ eines Zustands $s$ unter einer Strategie $\pi$ definiert als der Durchschnitt der diskontierten Belohnungen aller Teilsequenzen in $\partial_s$. Hierbei ist $\gamma$ der Diskontierungsfaktor und $rew^{\gamma}(O)$ die Funktion, die die diskontierte Belohnung einer Sequenz von Ereignissen berechnet.



#### **Beispiel 2.** <a name="b2"></a> Fortsetzung von Beispiel 1

Wir führen Beispiel 1 fort und betrachten den Zustand $s_{0,1}^1$. Insgesamt gibt es **6 (Teil-)Probeläufe** die in $s_{0,1}^1$ starten:

$$ \partial_{s_{0,1}^{1}} = \{ $$
$$((s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, \\ s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)), $$
$$((s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)),$$
$$((s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)), $$
$$((s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)),$$
$$((s_{0,1}^{1}, \text{move}, s_{0,1}^{1}, -1), (s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0)), $$
$$((s_{0,1}^{1}, \text{move}, s_{0,1}^{2}, -1), (s_{0,1}^{2}, \text{clean}, s_{0,0}^{2}, 10), (s_{0,0}^{2}, \text{move}, s_{0,0}^{1}, -1), (s_{0,0}^{1}, \text{charge}, s_{t}, 0))$$
$$\}$$

Dieser Zustand kommt (als erste Komponente einer Beobachtung) **einmal** in $O_1$, **zweimal** in $O_2$ und **dreimal** in $O_3$ vor. Deshalb erhalten wir weiterhin (für $\gamma = 0.9$)

$$U^\gamma_\partial(s_{0,1}^1 | \pi_{vc}) = \frac{1}{6} \left(3(-1+0.9 \cdot 10 + 0.9^2(-1)+0.9^3 \cdot 0)+2(-1+0.9(-1)+0.9^2 \cdot 10+0.9^3(-1)+0.9^4 \cdot 0)+( -1+0.9(-1)+0.9^2(-1)+0.9^3 \cdot 10+0.9^4(-1)+0.9^5 \cdot 0)\right) $$
$$\approx \frac{1}{6} \left(3 \cdot 7.19+2 \cdot 5.471+3.924\right) \approx 6.073$$


> **!!**
> 
> Bitte Beachte, dass die Werte $U^{\gamma}_\partial(s | \pi)$, die auf der Grundlage einer großen Anzahl von Probeläufen berechnet werden, sich den tatsächlichen Werten $U^{\gamma}_D(s | \pi)$ des generierenden Markov-Entscheidungsprozesses $D$ annähern bzw. gegen den tatsächlichen Werten $U^{\gamma}_D(s | \pi)$ konvergieren.
>
>Mit anderen Worten, wenn Sie eine ausreichend große Anzahl von Probeläufen durchführen, werden die geschätzten Werte $U^{\gamma}_\partial(s | \pi)$ immer näher an die tatsächlichen Werte $U^{\gamma}_D(s | \pi)$ herankommen.


Der Ansatz hat zwei grundlegende **Nachteile**:
1. Nur die Werte $U^{\gamma}_{\partial}(s | \pi)$ berechnet werden können, für die $s$ in einem Probelauf vorkommt. Ungesehene Zustände $s$ können also nicht approximiert werden.
2. die Approximation über $U^{\gamma}_{\partial}(s | \pi)$ nutzt nicht die Zusammenhänge zwischen den Zuständen. Der Nutzen eines Zustands $s$ ist direkt abhängig von den Nutzen seiner direkten Nachfolgezustände. Die Definition von $U^{\gamma}_{\partial}(s | \pi)$ als Durchschnitt der Nutzen aller beobachteten Probeläufe berechnet diese Nutzen für jeden Zustand $s$ unabhängig von allen anderen Zuständen.

In der Praxis bedeutet dies, dass eine Berechnung von $U^{\gamma}_D(s | \pi)$ über $U^{\gamma}_{\partial}(s | \pi)$ üblicherweise nur sehr langsam konvergiert. Während der erste Nachteil inhärent für alle Methoden des passiven Reinforcement Learnings ist, da wir eine feste Strategie annehmen und dadurch üblicherweise nicht alle möglichen Zustände sehen werden, können wir durch alternative Methoden den zweiten Nachteil umgehen.

Diese Methoden werden in den nächsten Abschnitten diskutiert.

## 4.2.2. Adaptive dynamische Programmierung <a name="4_2_2"></a>
---

---
<a name="up"></a>**`[Go Up!^](#up)**